In [1]:
from sage.rings.factorint import factor_trial_division


def sieve(K, qlimit, climit, ratio=1.1):
    """
    Input: K = GF(p), qlimit is the uppper bound for the factor base,
           climit is the limit for c_1's (should be much smaller than
           H = floor(sqrt(p)) + 1)
    Output: list containing pairs of elements of the extended factor 
            base and their respective discre logarithms 
    """
    
    p = len(K)

    # get the generator of K
    a = K.multiplicative_generator()
    
    H = floor(sqrt(p)) + 1
    J = H^2 - p
    
    # Get factor basis of all primes smaller than qlimit
    fb_primes = list(primes(qlimit))
    
    FB = fb_primes.copy()
    
    # Initialize extended factor base FB to fb_primes (starting with a)
    # note that starting with a ensures that when the vector mod p-1 is
    # normalized, the logarithms will be wrt to a
    if a in FB:
        FB.insert(0, FB.pop(FB.index(a)))
    
    if a not in FB:
        FB = [a] + fb_primes
    
    
    # Initialize A to 0 by 0, this will be the sparse matrix.
    A = []
    
    # Get logs of all factor basis primes
    logqs = [float(log(q, 2)) for q in fb_primes]
    
    for c1 in range(1, climit+1):
        
        # stop if ratio of relations to unknowns is sufficient default 1.1
        if len(A) / len(FB) >= ratio:
            break
        
        # initialize sieve
        sieve = [log(1) for i in range(1, climit+1)]
        
        den = H + c1          # denominator of relation
        num = -(J + c1*H)     # numerator
        
        for i in range(0, len(fb_primes)): #needs to be 0
            # For each prime q in factor base...
            q = fb_primes[i]
            logq = logqs[i]
            
            qpow = q
            while qpow <= qlimit:
                # For all powers qpow of q up to qlimit
                if den % qpow == 0:
                    break
                    
                c2 = num * inverse_mod(den, qpow) % qpow

                if c2 == 0:
                    c2 = qpow
                
                nextqpow = qpow * q
                
                # ensure c1 <= c2 to ignore redundant relations
                while c2 < c1:
                    c2 += qpow
                
                while c2 <= len(sieve):
                    
                    # Add logq into sieve for c2
                    sieve[c2-1] += float(logq)
                    
                    # Test higher powers of q if nextqpow is too large
                    if nextqpow > qlimit:
                        prod = (J + (c1 + c2)*H + c1*c2) % p
                        nextp = nextqpow
                        
                        while prod % nextp == 0:
                            # maybe here *****
                            sieve[c2-1] += float(logq)
                            nextp *= q
        
                    c2 += qpow        
                qpow = nextqpow 
                        
        # look in sieve for factorization
        rel = den * (H + 1)    
        rel_inc = H + c1       # add this to get next rel
        
        for c2 in range(1, len(sieve)+1):
            n = rel % p
            
            if abs(sieve[c2-1] - floor(log(n, 2))) < 1:
                
                fact = factor_trial_division(n, qlimit)
                
                r = 0
                # need to check if this is the full factorization
                
                if eval(str(fact)) == 1:
                    fact = [(1,2)]
                  
                if fact[-1][1] == 1:
                    if fact[-1][0] not in Primes():
                        r = fact[-1][0]
                        
                # check if biggest prime is less than qlimit
                if r == 0 and (fact[-1][0] < qlimit):
                    
                    # Include each H + c_i in extended factor basis at end
                    if H + c1 not in FB:
                        FB.append(H + c1)
                    if H + c2 not in FB:
                        FB.append(H + c2)
                            
                    # initialize a sparse row
                    row = [0 for b in range(0, len(FB))]

                    # now we update it

                    # Include relation (H + c1)*(H + c2) = fact
                    for pk, e in list(fact):
                        if pk == 1:
                            continue
                        # update A_i,j to e
                        j_1 = FB.index(pk)

                        # stick expont at j
                        row[j_1] = e
                        
                    if c1 == c2:
                        j_2 = FB.index(H + c1)
                        row[j_2] = -2
                        
                    else:
                        j_2 = FB.index(H + c1)
                        j_3 = FB.index(H + c2)
                        
                        row[j_2] = -1
                        row[j_3] = -1
                    
                    A.append(row)
                    
            rel += rel_inc
            
    # fill out the matrix with sparse entries
    for row in A:
        sparse_bit = [0]*(len(FB) - len(row))
        row += sparse_bit
    
    # this block of code is mostly formatting to
    # make a call to the Modular Solution method
    nrows = str(len(A))
    ncols = str(len(A[0]))
    B = str(A) 
    B = B.replace("(", "").replace(")", "")
    v = magma_free("C := Matrix(IntegerRing(),"+nrows+","+ncols+","+B+
                   "); \n ModularSolution(SparseMatrix(C), "+str(p-1)+");")
    v = str(v).replace("(", "").replace(")", "")
    v2 = [int(j) for j in str(v).split()]
    value_log = list(zip(FB, v2))
    return value_log
    


In [2]:
K = GF(103)
g = K.multiplicative_generator()
for v,l in sieve(K, 35, 27):
    if l is not 0:    # if the value is zero then the log was not found
        print("log_"+str(g)+"("+str(v)+") = "+str(l)+" mod 103")
        # double check
        print(str(g)+"^"+str(l)+" = "+ str(g^l) + " mod 103")
        print("")

log_5(5) = 1 mod 103
5^1 = 5 mod 103

log_5(2) = 44 mod 103
5^44 = 2 mod 103

log_5(3) = 39 mod 103
5^39 = 3 mod 103

log_5(7) = 4 mod 103
5^4 = 7 mod 103

log_5(11) = 61 mod 103
5^61 = 11 mod 103

log_5(13) = 72 mod 103
5^72 = 13 mod 103

log_5(17) = 70 mod 103
5^70 = 17 mod 103

log_5(19) = 80 mod 103
5^80 = 19 mod 103

log_5(23) = 24 mod 103
5^24 = 23 mod 103

log_5(29) = 86 mod 103
5^86 = 29 mod 103

log_5(31) = 57 mod 103
5^57 = 31 mod 103

log_5(12) = 25 mod 103
5^25 = 12 mod 103

log_5(14) = 48 mod 103
5^48 = 14 mod 103

log_5(15) = 40 mod 103
5^40 = 15 mod 103

log_5(16) = 74 mod 103
5^74 = 16 mod 103

log_5(21) = 43 mod 103
5^43 = 21 mod 103

log_5(38) = 22 mod 103
5^22 = 38 mod 103

log_5(20) = 0 mod 103
5^0 = 1 mod 103

log_5(26) = 1 mod 103
5^1 = 5 mod 103

log_5(35) = 5 mod 103
5^5 = 35 mod 103

log_5(22) = 3 mod 103
5^3 = 22 mod 103

log_5(33) = 100 mod 103
5^100 = 33 mod 103

log_5(34) = 12 mod 103
5^12 = 34 mod 103

log_5(24) = 69 mod 103
5^69 = 24 mod 103

log_5(25) = 